<a href="https://colab.research.google.com/github/siriusted/gym-dssat-notebooks/blob/master/First_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First steps with Gym-dssat

In this tutorial, we will show how to simply interact with the gym-dssat environment.

First, we need to install gym-dssat. This is done in the following code cell. 

**Note**: It might take a while

## Installation

In [ ]:
# install system dependencies
!apt-get update &> /dev/null
!apt-get install python3.7-dev python3.7-venv &> /dev/null
!wget http://pyyaml.org/download/libyaml/yaml-0.2.5.tar.gz &> /dev/null
!tar -xf yaml-0.2.5.tar.gz &> /dev/null 
!cd yaml-0.2.5 && ./configure &> /dev/null && make &> /dev/null && make install &> /dev/null


# install pip dependencies 
# only need gym==0.18.3, pyyaml > 5.1
!pip install gym==0.18.3 &> /dev/null
!pip install -U PyYAML &> /dev/null


# install pdi
!git clone https://gitlab.maisondelasimulation.fr/pdidev/pdi.git &> /dev/null
!mkdir pdi/build && cd pdi/build && cmake -DCMAKE_INSTALL_PREFIX='/opt/pdi' \
    -DDIST_PROFILE=User \
    -DCMAKE_VERBOSE_MAKEFILE=ON \
    -DBUILD_CFG_VALIDATOR=OFF \
    -DBUILD_DECL_HDF5_PLUGIN=OFF \
    -DBUILD_DECL_NETCDF_PLUGIN=OFF \
    -DBUILD_DECL_SION_PLUGIN=OFF \
    -DBUILD_FLOWVR_PLUGIN=OFF \
    -DBUILD_FORTRAN=ON \
    -DBUILD_FTI_PLUGIN=OFF \
    -DBUILD_HDF5_PARALLEL=OFF \
    -DBUILD_MPI_PLUGIN=OFF \
    -DBUILD_PYCALL_PLUGIN=ON \
    -DBUILD_PYTHON=ON \
    -DBUILD_SET_VALUE_PLUGIN=ON \
    -DBUILD_SERIALIZE_PLUGIN=ON \
    -DBUILD_SHARED_LIBS=ON \
    -DBUILD_TEST_PLUGIN=OFF \
    -DBUILD_TRACE_PLUGIN=ON \
    -DBUILD_USER_CODE_PLUGIN=ON \
    -DUSE_DEFAULT=EMBEDDED .. &> /dev/null

!cd pdi/build && make install &> /dev/null


# install dssat-pdi
!git clone --recurse-submodules https://gitlab.inria.fr/rgautron/gym_dssat_pdi.git &> /dev/null
!cd gym_dssat_pdi/dssat-csm-os && mkdir build && cd build \
    && cmake -DCMAKE_INSTALL_PREFIX='/opt/dssat_pdi' -DUSE_DEFAULT=EMBEDDED -DCMAKE_PREFIX_PATH='/opt/pdi/share/paraconf/cmake;/opt/pdi/share/pdi/cmake' .. &> /dev/null
!cd gym_dssat_pdi/dssat-csm-os/build && make &> /dev/null && make install &> /dev/null
!cd gym_dssat_pdi/dssat-csm-data &&  cp -r ./* /opt/dssat_pdi


# install gym-dssat
!cd gym_dssat_pdi/gym-dssat-pdi/ && pip install -e . &> /dev/null

Now, we're all set up to use gym-dssat!
<br/><br/>
To be able use gym-dssat properly, we need to run commands using `pdirun`. As a result, the source code for the rest of the tutorial has been collected into a script, which we will fetch next, then run.


In [ ]:
!wget https://raw.githubusercontent.com/siriusted/gym-dssat-notebooks/master/first_steps.py

Run example. You can take a look at `first_steps.py` in the file browser here on colab

In [ ]:
!/opt/pdi/bin/pdirun python first_steps.py

And, there you have it! Now, you can go ahead and keep playing with gym-dssat by editing `first_steps.py` and re-running the code cell above.